In [1]:
from tool import *
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter

In [2]:
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

# 변수 셋팅

In [3]:
with open('D:/Data/ODM GSCP.db', 'rb') as f:
    spdb = pickle.load(f)

spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
172,2024-01-15(W03),Latest26w,202401151014,1024819
173,2024-01-15(W03),Latest26w,202401151513,1024499
174,2024-01-15(W03),Latest26w,202401160856,1022183
175,2024-01-15(W03),Latest26w,202401170859,1022943
176,2024-01-22(W04),Final26w,202401231352,1076977
177,2024-01-22(W04),Final52w,202401240837,1372815
178,2024-01-22(W04),Latest26w,202401221317,1062138
179,2024-01-22(W04),Latest26w,202401221504,1076730
180,2024-01-22(W04),Latest26w,202401231003,1075980
181,2024-01-22(W04),Latest26w,202401260909,1076185


In [4]:
i = 187 # SP DB에서 선택할 index 넘버를 입력
fixed_week_period = 4

In [5]:
p_wk = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i, ]['Planweek']
ver = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i, ]['Category']
updated_time = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i, ]['Created_at']

sp = spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])
osp = sp[sp['From Site'].str.contains('AUO|TCL|QUANTA|TPV|WINGTECH|PEGATRON')].reset_index(drop=True)
osp = osp.drop(columns='Transport Mode')
sp = osp[osp['Frozen']=='Y'].copy()
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Biz Type,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP
1,2024-01-29(W05),202401310947,MNT,OS_AUO_CN_P,EU,EEBN,Regular,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Latest26w,Y,2024-01-01(W01),312
2,2024-01-29(W05),202401310947,MNT,OS_AUO_CN_P,EU,EEBN,Regular,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Latest26w,Y,2024-01-29(W05),312
5,2024-01-29(W05),202401310947,MNT,OS_AUO_CN_P,EU,EEBN,Regular,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,Y,2024-01-01(W01),195
6,2024-01-29(W05),202401310947,MNT,OS_AUO_CN_P,EU,EEBN,Regular,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,Y,2024-01-29(W05),195
7,2024-01-29(W05),202401310947,MNT,OS_AUO_CN_P,EU,EEBN,Regular,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,Y,2024-02-05(W06),429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2024-01-29(W05),202401310947,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,Regular,10A30Q-L.AQ26ML,10A30Q-L.AQ26ML,Latest26w,Y,2024-01-29(W05),52
2180,2024-01-29(W05),202401310947,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,Regular,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Latest26w,Y,2024-01-08(W02),360
2181,2024-01-29(W05),202401310947,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,Regular,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Latest26w,Y,2024-01-29(W05),228
2182,2024-01-29(W05),202401310947,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,Regular,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Latest26w,Y,2024-02-05(W06),310


# 1. GSCP DB에서 업체별 보정 SP를 엑셀로 만듬

In [6]:
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

index_col = ['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix']
sp = pd.merge(sp, ste[['To Site', 'Subsidiary', 'Country']], how='left')
sp = sp.fillna('TBA')
sp = sp.pivot_table('SP', index=index_col, columns='Week Name', aggfunc=sum).fillna(0).astype(int)

wk = sp.columns[0]
while wk < sp.columns[-1]:
    if get_weekname_from(wk, 1) not in sp.columns:
        sp.insert(sp.columns.get_loc(wk) + 1, get_weekname_from(wk, 1), 0)
    wk = get_weekname_from(wk, 1)

index_1 = sp.index
sp = sp.reset_index()
sp.insert(5, 'Category', 'SP')
cat_order = CategoricalDtype(['SP', 'SR', 'PO', 'SP_adj'], ordered=True) # 데이타프레임을 순서대로 표시하기 위해 category 데이타로 설정
sp['Category'] = sp['Category'].astype(cat_order)
sp = sp.set_index(index_col + ['Category'])

for i in sp.index:
    for item in ['SR', 'PO', 'SP_adj']:
        sp.loc[(i[0], i[1], i[2], i[3], i[4], i[5], i[6], item)] = 0
sp = sp.sort_index(level=0)

In [7]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f).reset_index()

sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr = sr[sr['Week Name'] < p_wk]
previous_weeklist = get_previous_weeklist(p_wk)
ship_result = sr.groupby('PO No.').sum('Ship')['Ship'].reset_index()
sr = sr[sr['Week Name'].isin(previous_weeklist)]
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, ste[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr = sr.fillna('TBA')
sr = sr.groupby(['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name']).sum('Ship')['Ship']
for fm, div, reg, sub, con, site, model, wk in sr.index:
    sp.loc[(fm, div, reg, sub, con, site, model, 'SR'), wk] = sr[(fm, div, reg, sub, con, site, model, wk)]

In [8]:
with open('D:/Data/ODM PO.db', 'rb') as f:
    po = pickle.load(f)

po = po[po['Cancel'] == 'N']
opo = pd.merge(po.reset_index(), ship_result, how='left').fillna(0)
opo['Week Name'] = opo['RSD'].apply(get_weekname)
end_week_of_fix = get_weekname_from(p_wk, fixed_week_period)
opo = opo[opo['Week Name'] <= end_week_of_fix]
opo['Ship'] = opo['Ship'].astype(int)
opo['Open'] = opo['PO'] - opo['Ship']
opo = opo[opo['Open'] > 0]
opo['To Site'] = opo['To Site'].replace(ship_to_map)
opo.loc[opo['Week Name'] < p_wk, 'Week Name'] = p_wk
opo = pd.merge(opo, ste[['Region', 'Subsidiary', 'Country', 'To Site']], how='left')
opo = opo.fillna('TBA')
opo = opo.groupby(['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name']).sum('Open')['Open']

for fm, div, reg, sub, con, site, model, wk in opo.index:
    sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), wk] = opo[(fm, div, reg, sub, con, site, model, wk)]

sp = sp.sort_index(axis=1).fillna(0).astype(int)

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\3145338216.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), wk] = opo[(fm, div, reg, sub, con, site, model, wk)]
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\3145338216.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), wk] = opo[(fm, div, reg, sub, con, site, model, wk)]
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\3145338216.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), wk] = opo[(fm, div, reg, sub, con, site, model, wk)]
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\3145338216.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), wk] = opo[(fm, div, reg, sub, con, site, mode

In [9]:
for fm, div, reg, sub, con, site, model in index_1:
    t1 = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), :get_weekname_from(p_wk, -1)].sum() - sp.loc[(fm, div, reg, sub, con, site, model, 'SR'), :get_weekname_from(p_wk, -1)].sum() # 과거 주차들의 SP값과 SR값의 차이를 구한 값
    t2 = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), p_wk] # 이번주차의 SP 값
    t3 = sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), p_wk] # 이번주차의 PO 값
    if (t1 == 0): # 과거 주차들의 SP 값과 SR 값의 차이가 없을 때는 실제 SP는 SR값이 되도록 함
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), :get_weekname_from(p_wk, -1)] = sp.loc[(fm, div, reg, sub, con, site, model, 'SR'), :get_weekname_from(p_wk, -1)]
        sum_sp = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), p_wk:].sum() # 확정구간의 SP의 합계
        sum_po = sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), p_wk:].sum() # 확정구간의 PO의 합계
        if sum_sp == sum_po: # 확정 구간 SP와 PO 의 합이 같다면,
            sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), p_wk:] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), p_wk:] # 실제 SP 는 SP 값으로 반영함
            reason = 'Normal Case'
            sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
        else: # 확정 구간 SP와 PO 의 값이 다른 경우라면,
            sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), p_wk:] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), p_wk:]
            reason = 'SP-PO is not matched in frozen period'
            sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
    elif (t1 > 0): # 과거 주차들의 SP값이 SR 값보다 클 경우에는, 
        reason = 'SP is greater than SR'
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj')] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP')]
        sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
    elif (t1 < 0) & ((t1 + t2 - t3) == 0): # 과거 주차들의 SR 값이 SP 값보다 크고, 그 큰 폭이 이번주차의 SP값에서 이번주차 SP에서 이번주차 PO값을 차감한 값과 동일하다면, 이런경우는 실제 선적했는데 GSCP는 선적을 못한 것으로 carry over한 경우임
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), :get_weekname_from(p_wk, -1)] = sp.loc[(fm, div, reg, sub, con, site, model, 'SR'), :get_weekname_from(p_wk, -1)] # 과거주차의 실제 SP값은 SR 값으로 함
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), p_wk] = t3 # 이번주차의 실제 SP값은 이번주차의 PO 값으로 함
        sum_sp = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), get_weekname_from(p_wk, 1):].sum()
        sum_po = sp.loc[(fm, div, reg, sub, con, site, model, 'PO'), get_weekname_from(p_wk, 1):].sum()
        if sum_sp == sum_po:
            sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), get_weekname_from(p_wk, 1):] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), get_weekname_from(p_wk, 1):]
        else:
            sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), get_weekname_from(p_wk, 1):] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), get_weekname_from(p_wk, 1):]
            reason = 'SD input delay and SP-PO is not matched in frozen period'
            sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
            continue
        reason = 'SD input delay'
        sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
    elif (t1 < 0) & ((t1 + t2 - t3) != 0): # 과거 주차 SR 값이 SP 보다 큰데, 이번주차 SP에서 이번주차 PO값을 차감한 값과 다르다면, SP-PO의 동기화에 뭔가 문제가 있는 것으로 간주함
        reason = 'Something wrong on SP-PO'
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), :get_weekname_from(p_wk, -1)] = sp.loc[(fm, div, reg, sub, con, site, model, 'SR'), :get_weekname_from(p_wk, -1)]
        sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), p_wk:] = sp.loc[(fm, div, reg, sub, con, site, model, 'SP'), p_wk:]
        sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\4133444825.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model, 'SP_adj'), p_wk] = t3 # 이번주차의 실제 SP값은 이번주차의 PO 값으로 함
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\4133444825.py:34: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\4133444825.py:39: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\4133444825.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  sp.loc[(fm, div, reg, sub, con, site, model), 'Type'] = reason
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12072\4133444825.py:23: PerformanceWarning: indexing past lexsort depth may 

In [10]:
sp = sp.reset_index().set_index(['From Site', 'Division', 'Region', 'Subsidiary', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category', 'Type'])
filename = f"{p_wk}_ODM_GSCP_SP_Analysis_{updated_time}_{ver}.xlsx" #이번주 forecast 파일을 저장할 이름 설정
with pd.ExcelWriter("D:/MNT/ODM_SP/SP_Analysis/"+filename) as writer:
    sp.to_excel(writer, sheet_name='SP Analysis', merge_cells=False, startrow=1)

In [11]:
filepath = 'D:/MNT/ODM_SP/SP_Analysis/' + filename
# filepath = get_filename()
wb = load_workbook(filepath)
                   
color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

# 첫번째 Sheet(SP Analysis)
sh = wb[wb.sheetnames[0]]
end_col = sh.max_column
end_row = sh.max_row
sh['A1'] = f'■ {p_wk} IT ODM Shipment Plan Analysis'
sh['A1'].font = big_size

for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
sh.column_dimensions['A'].width = 17
sh.column_dimensions['D'].width = 10
sh.column_dimensions['E'].width = 12
sh.column_dimensions['F'].width = 15
sh.column_dimensions['G'].width = 20
sh.column_dimensions['H'].width = 8
sh.column_dimensions['I'].width = 38
sh.freeze_panes = 'J3'
sh['I1'] = 'Sub Total'

for row in sh.iter_rows(min_row=3, min_col=10, max_row=end_row, max_col=end_col):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

for i in range(10, end_col+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{end_row})'

sh_names = wb.sheetnames
for sh in sh_names:
    wb[sh].sheet_view.showGridLines = False

wb.save(filepath)

# 2. 보정된 SP가 있는 Excel File을 불러와 DB에 저장

In [12]:
filepath = 'D:/MNT/ODM_SP/SP_Analysis/' + filename
ver = filename.split('.')[0].split('_')[-1]
updated_time = filename.split('.')[0].split('_')[-2]
p_wk = filename.split('.')[0].split('_')[0]

In [13]:
sp = pd.read_excel(filepath, sheet_name='SP Analysis', skiprows=1, keep_default_na=False, na_values=['', 'nan'])
sp = sp[sp['Category'] == 'SP_adj']
sp = sp.groupby(['From Site', 'Division', 'Region', 'To Site', 'Mapping Model.Suffix'])[get_weeklist(sp)].sum()
sp.columns.name = 'Week Name'
sp = sp.stack('Week Name')
sp = sp[sp > 0]
sp.name = 'SP'
sp = sp.reset_index()
pms_map = osp[['Mapping Model.Suffix', 'Rep PMS']].groupby('Mapping Model.Suffix')['Rep PMS'].max()
sp['Rep PMS'] = sp['Mapping Model.Suffix'].map(pms_map)
sp['Planweek'] = p_wk
sp['Created_at'] = updated_time
sp['Frozen'] = 'Y'
sp['Category'] = ver
osp = osp[osp['Frozen'] == 'N'].drop(columns='Biz Type')
sp = pd.concat([osp, sp]).reset_index(drop=True)

In [14]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    adjspdb = pickle.load(f)

In [15]:
adjspdb.shape

(301999, 12)

In [16]:
adjspdb = pd.concat([adjspdb, sp]).reset_index(drop=True)
adjspdb.shape

(304162, 12)

In [17]:
adjspdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
111,2024-01-15(W03),Latest26w,202401151014,1023539.0
112,2024-01-15(W03),Latest26w,202401151513,1023219.0
113,2024-01-15(W03),Latest26w,202401160856,1020903.0
114,2024-01-15(W03),Latest26w,202401170859,1021663.0
115,2024-01-22(W04),Final26w,202401231352,1075700.0
116,2024-01-22(W04),Final52w,202401240837,1362524.0
117,2024-01-22(W04),Latest26w,202401221317,1060858.0
118,2024-01-22(W04),Latest26w,202401221504,1075450.0
119,2024-01-22(W04),Latest26w,202401231003,1074703.0
120,2024-01-22(W04),Latest26w,202401260909,1074908.0


In [18]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'wb') as f:
    pickle.dump(adjspdb, f)